In [2]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

In [16]:
# Định nghĩa tên các cột theo tài liệu UCI
columns = [
    "Pregnancies",            # Số lần mang thai
    "Glucose",                # Nồng độ glucose huyết tương
    "BloodPressure",          # Huyết áp tâm trương
    "SkinThickness",          # Độ dày nếp gấp da (mm)
    "Insulin",                # Insulin huyết thanh (mu U/ml)
    "BMI",                    # Chỉ số khối cơ thể
    "DiabetesPedigreeFunction", # Hàm phả hệ tiểu đường (yếu tố di truyền)
    "Age",                    # Tuổi
    "Outcome"                 # Kết quả (0: Không mắc, 1: Mắc tiểu đường)
]
# Đọc dataset Pima Indians Diabetes
df = pd.read_csv("pima-indians-diabetes.csv", header=None, names=columns)

## Xử lý dữ liệu

### Xử lý dữ liệu trùng lặp

In [17]:
# Kiểm tra dòng trùng lặp
duplicates = df.duplicated()
print("Số dòng trùng lặp:", duplicates.sum())

# Xóa dòng trùng lặp
df_cleaned = df.drop_duplicates()
print("Kích thước sau khi xóa trùng lặp:", df_cleaned.shape)

Số dòng trùng lặp: 0
Kích thước sau khi xóa trùng lặp: (768, 9)


### Xác định ngưỡng sinh lý 

In [9]:
# NGƯỠNG RỘNG ĐỂ BẢO TOÀN DỮ LIỆU - CHỈ LỌC GIÁ TRỊ VÔ LÝ THỰC SỰ
physiological_ranges = {
    'Pregnancies': (0, 20),           # Số lần mang thai hợp lý
    'Glucose': (20, 300),             # Cho phép cả giá trị bệnh lý cao
    'BloodPressure': (40, 200),       # Bao gồm cả tăng huyết áp nặng
    'BMI': (10, 80),                  # Bao gồm cả béo phì bệnh lý
    'Age': (21, 100)                  # Tuổi hợp lý cho nghiên cứu
     # Với các chỉ số khác, có thể chỉ xử lý giá trị 0
}

### Phát hiện dữ liệu lỗi

In [10]:
def detect_physiological_errors(df, ranges):
    """Phát hiện dữ liệu ngoài ngưỡng sinh lý"""
    errors = {}
    
    for column, (min_val, max_val) in ranges.items():
        # Tìm các giá trị ngoài ngưỡng
        out_of_range = (df[column] < min_val) | (df[column] > max_val)
        error_count = out_of_range.sum()
        
        if error_count > 0:
            errors[column] = {
                'count': error_count,
                'min_actual': df[column].min(),
                'max_actual': df[column].max(),
                'problem_values': df[column][out_of_range].unique()
            }
    
    return errors

# Phát hiện lỗi
errors = detect_physiological_errors(df, physiological_ranges)
print("=== DỮ LIỆU LỖI PHÁT HIỆN ===")
for col, info in errors.items():
    print(f"{col}: {info['count']} giá trị lỗi")
    print(f"  - Range thực tế: {info['min_actual']} - {info['max_actual']}")
    print(f"  - Giá trị có vấn đề: {info['problem_values']}")

=== DỮ LIỆU LỖI PHÁT HIỆN ===
Glucose: 5 giá trị lỗi
  - Range thực tế: 0 - 199
  - Giá trị có vấn đề: [0]
BloodPressure: 39 giá trị lỗi
  - Range thực tế: 0 - 122
  - Giá trị có vấn đề: [ 0 30 24 38]
BMI: 11 giá trị lỗi
  - Range thực tế: 0.0 - 59.4
  - Giá trị có vấn đề: [0.]


### Xử lý dữ liệu lỗi

In [11]:
# Chiến lược: XÓA các dòng có dữ liệu lỗi nghiêm trọng
def clean_physiological_errors(df, ranges):
    """Xóa dòng có dữ liệu ngoài ngưỡng sinh lý"""
    
    # Tạo mask để đánh dấu dòng lỗi
    error_mask = pd.Series(False, index=df.index)
    
    # Các chỉ số QUAN TRỌNG cần xử lý nghiêm ngặt
    critical_columns = ['Glucose', 'BloodPressure', 'BMI', 'Age']
    
    for column in critical_columns:
        min_val, max_val = ranges[column]
        # Đánh dấu dòng có giá trị ngoài ngưỡng
        error_mask |= (df[column] < min_val) | (df[column] > max_val)
    
    print(f"Số dòng có dữ liệu lỗi nghiêm trọng: {error_mask.sum()}")
    
    # Giữ lại các dòng KHÔNG có lỗi
    df_clean = df[~error_mask].copy()
    
    return df_clean

# Xử lý dữ liệu lỗi
df_clean = clean_physiological_errors(df, physiological_ranges)

Số dòng có dữ liệu lỗi nghiêm trọng: 48


### Xử lý giá trị 0 bất thường

In [12]:
# Sau khi xử lý lỗi nghiêm trọng, xử lý giá trị 0
def handle_zero_values(df):
    """Xử lý giá trị 0 bất thường trong các chỉ số sinh học"""
    
    # Các cột không được phép có giá trị 0
    zero_sensitive_columns = ['Glucose', 'BloodPressure', 'BMI', 'SkinThickness']
    
    for col in zero_sensitive_columns:
        zero_mask = df[col] == 0
        if zero_mask.any():
            print(f"Phát hiện {zero_mask.sum()} giá trị 0 trong {col}")
            
            # Thay thế bằng median của cột (không bao gồm giá trị 0)
            median_val = df[col][df[col] > 0].median()
            df.loc[zero_mask, col] = median_val
            print(f"  → Đã thay thế bằng median: {median_val:.2f}")
    
    return df

# Xử lý giá trị 0
df_final = handle_zero_values(df_clean)

Phát hiện 192 giá trị 0 trong SkinThickness
  → Đã thay thế bằng median: 29.00


### Kiểm tra kết quả

In [13]:
print("=== KẾT QUẢ SAU XỬ LÝ ===")
print(f"Dữ liệu gốc: {len(df)} dòng")
print(f"Sau xử lý: {len(df_final)} dòng")
print(f"Tỷ lệ giữ lại: {len(df_final)/len(df)*100:.1f}%")

# Kiểm tra lại ngưỡng sinh lý
final_errors = detect_physiological_errors(df_final, physiological_ranges)
if not final_errors:
    print("✅ Tất cả dữ liệu đều trong ngưỡng sinh lý hợp lý")
else:
    print("❌ Vẫn còn dữ liệu lỗi:")
    for col, info in final_errors.items():
        print(f"  - {col}: {info['count']} lỗi")

=== KẾT QUẢ SAU XỬ LÝ ===
Dữ liệu gốc: 767 dòng
Sau xử lý: 719 dòng
Tỷ lệ giữ lại: 93.7%
✅ Tất cả dữ liệu đều trong ngưỡng sinh lý hợp lý
